In [1]:
import numpy as np
import tensorflow as tf
import scipy.io
import tensorly as ts
import os
os.chdir("G:/jupyter/Guangzhou-data-set")
tensor = scipy.io.loadmat('tensor.mat')
tensor = tensor['tensor']
# random_matrix = scipy.io.loadmat('Guangzhou-data-set/random_matrix.mat')
# random_matrix = random_matrix['random_matrix']
# random_tensor = scipy.io.loadmat('Guangzhou-data-set/random_tensor.mat')
# random_tensor = random_tensor['random_tensor']

tensor = np.array(tensor)
print(tensor.shape)

Using numpy backend.


(214, 61, 144)


In [2]:
missing_rate = 0.5
condition = np.random.binomial(1, 1-missing_rate, np.shape(tensor)[0]*np.shape(tensor)[1]*np.shape(tensor)[2])
condition = np.reshape(condition,(np.shape(tensor)[0],np.shape(tensor)[1],np.shape(tensor)[2]))
mask = np.where(condition<1,0.,1.)
print(mask.shape)
sparse_tensor = np.multiply(tensor, mask)
print(sparse_tensor.shape)

mean = (np.sum(sparse_tensor)*1.0/np.sum(mask))
init_tensor = sparse_tensor
init_tensor[init_tensor==0] = mean
print(init_tensor.shape)

(214, 61, 144)
(214, 61, 144)
(214, 61, 144)


In [3]:
def unfold(X_tensor, dim, mode):
    order_mode_dim = [mode-1]
    order_others = list(range(0,mode-1))+list(range(mode,len(dim)))
    order_adjust = order_mode_dim + order_others[::-1]
    X_per = tf.transpose(X_tensor,order_adjust)
    X_unfold = tf.reshape(X_per,[dim[mode-1],-1])
    return X_unfold

def fold(X,dim,mode):
    order_mode_dim = [mode-1]
    order_others = list(range(0,mode-1))+list(range(mode,len(dim)))
    order_adjust = order_mode_dim + order_others[::-1]
    dim_set = [dim[i] for i in order_adjust]
    order_adjust=[dim_set.index(dim[i]) for i in range(len(dim))]
    X_tensor = tf.transpose(tf.reshape(X,dim_set),order_adjust)
    return X_tensor

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape = size, stddev = xavier_stddev)

def Kron_TF(A, B):
    A_shape = A.get_shape()
    B_shape = B.get_shape()
    
    for i in range(A_shape[0]):
        for j in range(A_shape[1]):
            if j==0:
                temp = tf.squeeze(A[i,j]*B)
             
            else:
                temp = tf.concat([temp,tf.squeeze(A[i,j]*B)],1)
        if i==0:
            result = temp
        else:
            result = tf.concat([result,temp],0)
    return result

In [4]:
dim = tensor.shape
rank1 = 150
rank2 = 40
rank3 = 100
tf.reset_default_graph()

init_tensor_holder = tf.placeholder(tf.float32, shape=[dim[0], dim[1], dim[2]]) 
sparse_tensor_holder = tf.placeholder(tf.float32, shape=[dim[0], dim[1], dim[2]])    
tensor_holder = tf.placeholder(tf.float32, shape=[dim[0], dim[1], dim[2]])  
mask_tensor_holder =  tf.placeholder(tf.float32, shape=[dim[0], dim[1], dim[2]])

sess = tf.Session()

init_unfold1 = unfold(init_tensor_holder,[dim[0], dim[1], dim[2]],1)
su, uu, vu = tf.svd(init_unfold1)

init_unfold2 = unfold(init_tensor_holder,[dim[0], dim[1], dim[2]],2)
sv, uv, vv = tf.svd(init_unfold2)

init_unfold3 = unfold(init_tensor_holder,[dim[0], dim[1], dim[2]],3)
sw, uw, vw = tf.svd(init_unfold3)

uu = uu[:,0:rank1]
uv = uv[:,0:rank2]
uw = uw[:,0:rank3]

# estimate core tensor
G_1_ = tf.matmul(tf.matmul(tf.transpose(uu),init_unfold1),Kron_TF(uw,uv))
g = fold(G_1_, [rank1,rank2,rank3], 1)

# initialzation construction
G_, U_,V_,W_,init_unfold1_=sess.run([g,uu,uv,uw,init_unfold1], feed_dict={init_tensor_holder:init_tensor})


G = tf.Variable(G_)
U = tf.Variable(U_)
V = tf.Variable(V_)
W = tf.Variable(W_)

# weight_initer = tf.truncated_normal_initializer(mean=mean, stddev=0.1)
# with tf.variable_scope("tucker_element", reuse=tf.AUTO_REUSE ):
#     G = tf.get_variable(name="G", dtype=tf.float32, shape=[rank1, rank2, rank3], initializer=weight_initer)
#     U = tf.get_variable(name="U", dtype=tf.float32, shape=[dim[0], rank1], initializer=weight_initer)
#     V = tf.get_variable(name="V", dtype=tf.float32, shape=[dim[1], rank2], initializer=weight_initer)
#     W = tf.get_variable(name="W", dtype=tf.float32, shape=[dim[2], rank3], initializer=weight_initer)

# G = tf.Variable(xavier_init([rank1, rank2, rank3]))
# U = tf.Variable(xavier_init([dim[0], rank1]))
# V = tf.Variable(xavier_init([dim[1], rank2]))
# W = tf.Variable( xavier_init([dim[2], rank3]))

# lembdaG = tf.constant(0.1)
# lembdaU = tf.constant(0.1)
# lembdaV = tf.constant(0.1)
# lembdaW = tf.constant(0.1)

Train_Param=[G,U,V,W]


In [5]:
# Y=G X_1 U X_2 V X_3 W
# T=A X_1 B => Y(1)=BX(1)

In [6]:
# Tucker decomposition
# G X_1 U
dim_G = [rank1,rank2,rank3]
G_unfold1 = unfold(G,dim_G,1)
mat1 = tf.nn.elu(tf.matmul(U,G_unfold1))

# X_2 V
dim_mat1 = [dim[0], rank2, rank3]
ten1 = fold(mat1, dim_mat1, 1)
dim_ten1 = ten1.get_shape()
ten1_unfold2 = unfold(ten1, dim_ten1, 2)
mat2 = tf.nn.elu(tf.matmul(V, ten1_unfold2))

# X_3 W
dim_mat2 = [dim[0], dim[1], rank3]
ten2 = fold(mat2, dim_mat2, 2)
dim_ten2 = ten2.get_shape()
ten2_unfold3 = unfold(ten2, dim_ten2, 3)
mat3 = tf.nn.elu(tf.matmul(W, ten2_unfold3))

# reshape back to tensor
dim_mat3 = [dim[0],dim[1],dim[2]]
impute_tensor = fold(mat3, dim_mat3, 3)


In [7]:
# loss function

loss = tf.reduce_mean((mask_tensor_holder * tensor_holder - mask_tensor_holder * impute_tensor) ** 2)
loss_evaluation = tf.reduce_sum(( (1.-mask_tensor_holder)*tensor_holder - (1.-mask_tensor_holder) * impute_tensor)  ** 2) / (tf.math.reduce_sum(1.-mask_tensor_holder))
loss_all = tf.reduce_mean((tensor_holder - impute_tensor) ** 2)
# r_g = tf.contrib.layers.l1_regularizer(scale=0.00001, scope=None)
# weights = tf.trainable_variables()
# r1 = tf.contrib.layers.apply_regularization(r_g, weights)
loss_reg = loss

solver = tf.train.AdamOptimizer(0.0004).minimize(loss_reg, var_list=Train_Param)

In [8]:
sess.run(tf.global_variables_initializer())

for i in range(50000):
    _, loss_reg_,it = sess.run([solver, loss_reg, impute_tensor], feed_dict = 
                           {mask_tensor_holder: mask, sparse_tensor_holder: sparse_tensor, 
                            tensor_holder:tensor})
    if i % 100==0:
        imputation_result,loss_eval,loss_train,loss_all_ = sess.run([impute_tensor,loss_evaluation,loss,loss_all], feed_dict =
                                     {mask_tensor_holder: mask, sparse_tensor_holder: sparse_tensor, 
                                      tensor_holder:tensor})
        
        print('epoch = {:}'.format(i)
        print('TRAIN RMSE = {:.6}'.format(np.sqrt(loss_train)))
        print('TEST RMSE = {:.6}'.format(np.sqrt(loss_eval)))
        print('Overall RMSE = {:.6}'.format(np.sqrt(loss_all_)))
        print('-------------------------------------------------')

        

TRAIN RMSE = 28.0218
TEST RMSE = 39.9584
Overall RMSE = 39.7876
-------------------------------------------------
TRAIN RMSE = 11.645
TEST RMSE = 18.0772
Overall RMSE = 17.2883
-------------------------------------------------
TRAIN RMSE = 4.36175
TEST RMSE = 8.00027
Overall RMSE = 7.14164
-------------------------------------------------
TRAIN RMSE = 4.15625
TEST RMSE = 7.71712
Overall RMSE = 6.85776
-------------------------------------------------
TRAIN RMSE = 4.01662
TEST RMSE = 7.50112
Overall RMSE = 6.65175
-------------------------------------------------
TRAIN RMSE = 3.89442
TEST RMSE = 7.29712
Overall RMSE = 6.46304
-------------------------------------------------
TRAIN RMSE = 3.78523
TEST RMSE = 7.103
Overall RMSE = 6.28774
-------------------------------------------------
TRAIN RMSE = 3.68712
TEST RMSE = 6.91845
Overall RMSE = 6.12454
-------------------------------------------------
TRAIN RMSE = 3.59698
TEST RMSE = 6.74277
Overall RMSE = 5.97111
---------------------------

TRAIN RMSE = 2.1193
TEST RMSE = 3.80604
Overall RMSE = 3.42475
-------------------------------------------------
TRAIN RMSE = 2.11249
TEST RMSE = 3.80162
Overall RMSE = 3.41809
-------------------------------------------------
TRAIN RMSE = 2.10569
TEST RMSE = 3.79723
Overall RMSE = 3.41144
-------------------------------------------------
TRAIN RMSE = 2.09893
TEST RMSE = 3.79301
Overall RMSE = 3.40493
-------------------------------------------------
TRAIN RMSE = 2.09222
TEST RMSE = 3.78894
Overall RMSE = 3.39853
-------------------------------------------------
TRAIN RMSE = 2.08553
TEST RMSE = 3.78491
Overall RMSE = 3.39217
-------------------------------------------------
TRAIN RMSE = 2.07887
TEST RMSE = 3.78096
Overall RMSE = 3.38587
-------------------------------------------------
TRAIN RMSE = 2.07225
TEST RMSE = 3.77709
Overall RMSE = 3.37964
-------------------------------------------------
TRAIN RMSE = 2.06566
TEST RMSE = 3.77373
Overall RMSE = 3.37373
-------------------------

TRAIN RMSE = 1.76428
TEST RMSE = 3.66989
Overall RMSE = 3.13716
-------------------------------------------------
TRAIN RMSE = 1.76103
TEST RMSE = 3.66953
Overall RMSE = 3.13513
-------------------------------------------------
TRAIN RMSE = 1.75783
TEST RMSE = 3.66908
Overall RMSE = 3.13307
-------------------------------------------------
TRAIN RMSE = 1.7546
TEST RMSE = 3.66872
Overall RMSE = 3.13105
-------------------------------------------------
TRAIN RMSE = 1.75155
TEST RMSE = 3.66927
Overall RMSE = 3.12966
-------------------------------------------------
TRAIN RMSE = 1.74829
TEST RMSE = 3.66801
Overall RMSE = 3.12709
-------------------------------------------------
TRAIN RMSE = 1.74516
TEST RMSE = 3.6677
Overall RMSE = 3.12516
-------------------------------------------------
TRAIN RMSE = 1.74256
TEST RMSE = 3.66636
Overall RMSE = 3.12293
-------------------------------------------------
TRAIN RMSE = 1.73898
TEST RMSE = 3.66702
Overall RMSE = 3.12131
--------------------------

TRAIN RMSE = 1.58342
TEST RMSE = 3.67089
Overall RMSE = 3.03974
-------------------------------------------------
TRAIN RMSE = 1.5816
TEST RMSE = 3.6711
Overall RMSE = 3.03892
-------------------------------------------------
TRAIN RMSE = 1.58038
TEST RMSE = 3.67345
Overall RMSE = 3.0397
-------------------------------------------------
TRAIN RMSE = 1.57799
TEST RMSE = 3.67156
Overall RMSE = 3.03731
-------------------------------------------------
TRAIN RMSE = 1.57618
TEST RMSE = 3.67174
Overall RMSE = 3.03648
-------------------------------------------------
TRAIN RMSE = 1.5744
TEST RMSE = 3.67199
Overall RMSE = 3.03572
-------------------------------------------------
TRAIN RMSE = 1.57469
TEST RMSE = 3.67118
Overall RMSE = 3.03538
-------------------------------------------------
TRAIN RMSE = 1.57087
TEST RMSE = 3.67259
Overall RMSE = 3.03425
-------------------------------------------------
TRAIN RMSE = 1.56921
TEST RMSE = 3.67229
Overall RMSE = 3.03321
----------------------------

KeyboardInterrupt: 